In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import re

In [139]:
# get_match_summary()
url

'https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/match-schedule-fixtures-and-results'

In [228]:
def get_match_summary(url, aband):
    match_summary={}
    html_content=requests.get(url).text
    soup=BeautifulSoup(html_content, 'lxml')
    soup_2 = soup.find_all('div', attrs={'class':'ds-w-full ds-bg-fill-content-prime ds-overflow-hidden ds-rounded-xl ds-border ds-border-line'})[0]
    venue_date=soup_2.find_all('div', attrs={'class':'ds-px-4 ds-py-3 ds-border-b ds-border-line'})[0].find('div', attrs={'class':'ds-text-tight-m ds-font-regular ds-text-ui-typo-mid'}).text.split(',')
#     print(len(venue_date), venue_date)
    if len(venue_date)==5:
        venue_date=venue_date[1:4]       
    else:
        venue_date=venue_date[2:5]  
    venue=venue_date[0]
    date=venue_date[1].split(' ')[2]+'-'+venue_date[1].split(' ')[1][:3]+'-'+venue_date[2][1:]
    win=soup_2.find_all('div', attrs={'class':'ds-w-2/3'})
    if len(win)==0:
        winner='none'
        margin='none'
    else:
        win=win[0].find('p', attrs={'class':'ds-text-tight-m ds-font-regular ds-truncate ds-text-typo-title'}).text
        win=win.split('(')[0]
        if(win[-1]==" "):
            win=win[:-1]
        margin=win.split(' ')[-2]+" "+win.split(' ')[-1]
        if win.split(' ')[2]!='by':
            winner=win.split(' ')[0]+" "+win.split(' ')[1]
        else:
            winner=win.split(' ')[0]
    match_summary['Team 1']=aband[0].text
    match_summary['Team 2']=aband[2].text
    match_summary['Winner']=winner
    match_summary['Margin']=margin
    match_summary['Ground']=venue
    match_summary['Date']=date
    return match_summary

# print(get_match_summary('https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/pakistan-vs-south-africa-36th-match-group-2-1298170/full-scorecard'))
# print(get_match_summary('https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/new-zealand-vs-pakistan-1st-semi-final-1298177/full-scorecard'))

In [227]:
def match_details(url):
    match_id = int(url.split('/')[5].split('-')[-1])
    html_content=requests.get(url).text
    soup=BeautifulSoup(html_content, 'lxml')
    soup_1 = soup.find_all('div', attrs={'class':'ds-mt-3'})[1]
    batting_details_per_match=[]
    bowling_details_per_match=[]
    aband=soup.find_all('div', attrs={'class':'ds-text-compact-xxs ds-p-2 ds-px-4 lg:ds-py-3'})[0].find_all('span')
    match_summary=get_match_summary(url, aband)
    if aband[2].text=="Match abandoned without a ball bowled":
        print(aband[0].text+' vs '+aband[1].text+' : '+aband[2].text)
        return batting_details_per_match
    match_name=aband[0].text+' vs '+aband[2].text
    match_summary['Match Id']=match_id
    print(match_name, match_summary['Winner'], match_summary['Margin'])
    for i in [0,2]:
        batting_table=soup_1.find_all('tbody')[i]
        j=1
        for row in batting_table.find_all('tr', {'class': ""}):
            batsman={}
            batsman['Match']=match_name
            batsman['Player']=row.find('td', {'class': ['ds-w-0', 'ds-whitespace-nowrap', 'ds-min-w-max']}).text.replace(u"\xa0", "").replace(u"(c)","").replace(u"†", "")
            if batsman['Player']=='TOTAL':
                break
            batsman['Batting_position']=j
            j+=1
            stats=row.find_all('td', {'class': ['ds-w-0', 'ds-whitespace-nowrap', 'ds-min-w-max', 'ds-text-right']})
            batsman['Team']=aband[i].text
            if stats[1].text=='not out':
                batsman['N/O']='not out'
            else:
                batsman['N/O']='out'
            batsman['Runs']=float(stats[2].text)
            batsman['Balls Played']=float(stats[3].text)
            minutes=stats[4].text
            if minutes=='-':
                batsman['Minutes Played']=0
            else:
                batsman['Minutes Played']=float(minutes)
            batsman['Fours']=float(stats[5].text)
            batsman['Sixes']=float(stats[6].text)
            strike=stats[7].text
            if strike=='-':
                batsman['Strike Rate']=0
            else:
                batsman['Strike Rate']=float(strike)
            batsman['Match Id']=match_id
            batting_details_per_match.append(batsman)
    
    for i in [1,3]:
        bowling_table=soup_1.find_all('tbody')[i]
        for row in bowling_table.find_all('tr', {'class': ""}):
            bowler={}
            details=row.find_all('td')
            bowler['Match']=match_name
            bowler['Team']=aband[3-i].text
            bowler['Player']=details[0].text
            bowler['Overs']=float(details[1].text)
            bowler['Maiden']=float(details[2].text)
            bowler['Runs']=float(details[3].text)
            bowler['Wickets']=float(details[4].text)
            bowler['Economy']=float(details[5].text)
            bowler['0s']=float(details[6].text)
            bowler['4s']=float(details[7].text)
            bowler['6s']=float(details[8].text)
            bowler['Wide']=float(details[9].text)
            bowler['No ball']=float(details[10].text)
            bowler['Match Id']=match_id
            bowling_details_per_match.append(bowler)
    return batting_details_per_match, bowling_details_per_match, match_summary

In [231]:
batting_details_season=[]
bowling_details_season=[]
match_summaries=[]
url='https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/match-schedule-fixtures-and-results'
html_content=requests.get(url).text
soup=BeautifulSoup(html_content, 'lxml')
soup_1 = soup.find_all('div', attrs={'class':'ds-p-0'})[0]
soup_2 = soup_1.find_all('div', attrs={'class':'ds-grow ds-px-4 ds-border-r ds-border-line-default-translucent'})
for g in soup_2:
#     print('https://www.espncricinfo.com'+g.find_all('a')[0].get('href'))
    details_per_match=match_details('https://www.espncricinfo.com'+g.find_all('a')[0].get('href'))
    if len(details_per_match)==0:
        continue
    batting_details_season+=details_per_match[0]
    bowling_details_season+=details_per_match[1]
    match_summaries.append(details_per_match[2]) 

Namibia vs Sri Lanka Namibia 55 runs
United Arab Emirates vs Netherlands Netherlands 3 wickets
Scotland vs West Indies Scotland 42 runs
Zimbabwe vs Ireland Zimbabwe 31 runs
Namibia vs Netherlands Netherlands 5 wickets
Sri Lanka vs United Arab Emirates Sri Lanka 79 runs
Scotland vs Ireland Ireland 6 wickets
West Indies vs Zimbabwe West Indies 31 runs
Sri Lanka vs Netherlands Sri Lanka 16 runs
United Arab Emirates vs Namibia U.A.E. 7 runs
West Indies vs Ireland Ireland 9 wickets
Scotland vs Zimbabwe Zimbabwe 5 wickets
New Zealand vs Australia New Zealand 89 runs
Afghanistan vs England England 5 wickets
Ireland vs Sri Lanka Sri Lanka 9 wickets
Pakistan vs India India 4 wickets
Bangladesh vs Netherlands Bangladesh 9 runs
Zimbabwe vs South Africa none none
Sri Lanka vs Australia Australia 7 wickets
Ireland vs England Ireland 5 runs
Afghanistan vs New Zealand : Match abandoned without a ball bowled
South Africa vs Bangladesh South Africa 104 runs
India vs Netherlands India 56 runs
Zimbabwe v

In [232]:
bat=pd.DataFrame(batting_details_season)
bat.to_csv('Batting_details_throughout_the_season.csv')
bowl=pd.DataFrame(bowling_details_season)
bowl.to_csv('Bowling_details_throughout_the_season.csv')
matches=pd.DataFrame(match_summaries)
display(bat, bowl, matches)
matches.to_csv('Match_Summaries.csv')

,Match,Player,Batting_position,Team,N/O,Runs,Balls Played,Minutes Played,Fours,Sixes,Strike Rate,Match Id
0,Namibia vs Sri Lanka,Michael van Lingen,1,Namibia,out,3.0,6.0,7.0,0.0,0.0,50.00,1298135
1,Namibia vs Sri Lanka,Divan la Cock,2,Namibia,out,9.0,9.0,15.0,1.0,0.0,100.00,1298135
2,Namibia vs Sri Lanka,Jan Nicol Loftie-Eaton,3,Namibia,out,20.0,12.0,18.0,1.0,2.0,166.66,1298135
3,Namibia vs Sri Lanka,Stephan Baard,4,Namibia,out,26.0,24.0,49.0,2.0,0.0,108.33,1298135
4,Namibia vs Sri Lanka,Gerhard Erasmus,5,Namibia,out,20.0,24.0,30.0,0.0,0.0,83.33,1298135
...,...,...,...,...,...,...,...,...,...,...,...,...
694,Pakistan vs England,Phil Salt,3,England,out,10.0,9.0,16.0,2.0,0.0,111.11,1298179
695,Pakistan vs England,Ben Stokes,4,England,out,52.0,49.0,81.0,5.0,1.0,106.12,1298179
696,Pakistan vs England,Harry Brook,5,England,out,20.0,23.0,36.0,1.0,0.0,86.95,1298179
697,Pakistan vs England,Moeen Ali,6,England,out,19.0,13.0,30.0,3.0,0.0,146.15,1298179


,Match,Team,Player,Overs,Maiden,Runs,Wickets,Economy,0s,4s,6s,Wide,No ball,Match Id
0,Namibia vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4.0,0.0,23.0,1.0,5.75,7.0,0.0,0.0,2.0,0.0,1298135
1,Namibia vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4.0,0.0,39.0,1.0,9.75,6.0,3.0,1.0,2.0,0.0,1298135
2,Namibia vs Sri Lanka,Sri Lanka,Pramod Madushan,4.0,0.0,37.0,2.0,9.25,6.0,3.0,1.0,0.0,0.0,1298135
3,Namibia vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4.0,0.0,36.0,1.0,9.00,7.0,3.0,1.0,1.0,0.0,1298135
4,Namibia vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4.0,0.0,27.0,1.0,6.75,8.0,1.0,1.0,0.0,0.0,1298135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Pakistan vs England,Pakistan,Naseem Shah,4.0,0.0,30.0,0.0,7.50,15.0,3.0,1.0,1.0,0.0,1298179
496,Pakistan vs England,Pakistan,Haris Rauf,4.0,0.0,23.0,2.0,5.75,13.0,3.0,0.0,1.0,0.0,1298179
497,Pakistan vs England,Pakistan,Shadab Khan,4.0,0.0,20.0,1.0,5.00,10.0,1.0,0.0,0.0,0.0,1298179
498,Pakistan vs England,Pakistan,Mohammad Wasim,4.0,0.0,38.0,1.0,9.50,5.0,5.0,0.0,2.0,0.0,1298179


,Team 1,Team 2,Winner,Margin,Ground,Date,Match Id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,16-Oct-2022,1298135
1,United Arab Emirates,Netherlands,Netherlands,3 wickets,Geelong,16-Oct-2022,1298136
2,Scotland,West Indies,Scotland,42 runs,Hobart,17-Oct-2022,1298137
3,Zimbabwe,Ireland,Zimbabwe,31 runs,Hobart,17-Oct-2022,1298138
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,18-Oct-2022,1298139
5,Sri Lanka,United Arab Emirates,Sri Lanka,79 runs,Geelong,18-Oct-2022,1298140
6,Scotland,Ireland,Ireland,6 wickets,Hobart,19-Oct-2022,1298141
7,West Indies,Zimbabwe,West Indies,31 runs,Hobart,19-Oct-2022,1298142
8,Sri Lanka,Netherlands,Sri Lanka,16 runs,Geelong,20-Oct-2022,1298143
9,United Arab Emirates,Namibia,U.A.E.,7 runs,Geelong,20-Oct-2022,1298144
